### Import Libraries

In [42]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Import Useful Functions

In [43]:
parent_directory = os.path.join(os.environ["BLACK_BOX"])
evals_path = os.path.join(parent_directory, "experiments/results/evaluation_statistics")
sys.path.append(parent_directory)

from experiments.utils.validation_utils import load_eval_from_csv

### Get Trained Agent Model Paths

In [44]:
# evaluation RL model folders located in ./experiments/results/evaluation_statistics folder
folder_name_1 = "evaluation_PPOTrainer_highway_environment:highway-environment-v0_2022-08-07_21-22-18mdgw9lf4_Agent0"

### Get All Files In the Directory

In [45]:
eval_directory_1 = os.path.join(evals_path, folder_name_1)
episode_evals_results_1 = [eval_directory_1 + "/" + f for f in os.listdir(eval_directory_1) if os.path.isfile(os.path.join(eval_directory_1, f))]

### Return All Statistics for Each Episode

In [46]:
full_eps_results = []
eps_rewards = []

for eps_stat in episode_evals_results_1:
    
    stat_df = load_eval_from_csv(file_name=eps_stat)
    full_eps_results.append(stat_df)
    
    eps_avg_reward = stat_df["ego_rewards"].mean(skipna = True)
    eps_sum_reward = stat_df["ego_rewards"].sum(skipna = True)
    eps_rewards.append(eps_sum_reward)

### Plot Results

In [47]:
y_values = eps_rewards
x_values = list(range(1, len(y_values) + 1, 1))

In [ ]:
plt.title("Evaluation Episode Reward")
plt.plot(x_values, y_values, color="red")


plt.show()

: 